<a href="https://colab.research.google.com/github/ElinorZhang0/ElinorZhang0/blob/main/NTF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import
import pandas as pd
from pprint import pprint
from pandas import Series, DataFrame
from pathlib import Path
import glob
from numpy.lib import type_check
import os
import pyarrow.parquet as pq

import cv2
from google.colab.patches import cv2_imshow
import numpy as np

In [ ]:
# connect google drive 
from google.colab import drive
# If drive is not already mounted: 
if not Path('/content/drive').exists():
  drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# set dictory
os.chdir("/content/drive/MyDrive/Assets")

In [ ]:
ls

 Arched.png             face.png            Mountains.png      RedMask.png
 BikerJacket.png        FormalShirt.png     Neck.png           Red.png
 Blank.png              Glasses.png         NormalHair.png     SlickedHair.png
 BlueMask.png           GrayHat.png         Normal.png         SmilingMouth.png
 Blue.png               GrinningMouth.png   Nose.png           Smiling.png
'Border and text.png'   LightBlue.png       NuclearPlant.png   Sunglasses.png
 Coat.png               LightGrayCoat.png   RedHat.png         TealLazer.png
 Ear.png                LightRed.png        RedLazer.png       T-Shirt.png


In [ ]:
# connect spreadsheet
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

# read spreadsheet
worksheet = gc.open('Copy of NFT Combinations').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
test = pd.DataFrame.from_records(rows)

[['Background', 'Neck', 'Innerwear', 'Outerwear', 'Ear', 'Face', 'Nose', 'Eyebrows', 'Eyewear', 'Border and text', 'Hair', 'Mouth', 'Mask', 'LazerEyes', 'Hat'], ['NuclearPlant.png', 'Neck.png', 'T-Shirt.png', 'BikerJacket.png', 'Ear.png', 'face.png', 'Nose.png', 'Arched.png', 'Glasses.png', 'Border and text.png', 'SlickedHair.png', 'GrinningMouth.png', 'BlueMask.png', 'TealLazer.png', 'GrayHat.png'], ['Mountains.png', 'Neck.png', 'T-Shirt.png', 'BikerJacket.png', 'Ear.png', 'face.png', 'Nose.png', 'Arched.png', 'Glasses.png', 'Border and text.png', 'SlickedHair.png', 'GrinningMouth.png', 'BlueMask.png', 'TealLazer.png', 'GrayHat.png'], ['LightRed.png', 'Neck.png', 'T-Shirt.png', 'BikerJacket.png', 'Ear.png', 'face.png', 'Nose.png', 'Arched.png', 'Glasses.png', 'Border and text.png', 'SlickedHair.png', 'GrinningMouth.png', 'BlueMask.png', 'TealLazer.png', 'GrayHat.png'], ['LightBlue.png', 'Neck.png', 'T-Shirt.png', 'BikerJacket.png', 'Ear.png', 'face.png', 'Nose.png', 'Arched.png', 'Gla

In [ ]:
print(test.shape)
test.head()

(2665, 15)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,Background,Neck,Innerwear,Outerwear,Ear,Face,Nose,Eyebrows,Eyewear,Border and text,Hair,Mouth,Mask,LazerEyes,Hat
1,NuclearPlant.png,Neck.png,T-Shirt.png,BikerJacket.png,Ear.png,face.png,Nose.png,Arched.png,Glasses.png,Border and text.png,SlickedHair.png,GrinningMouth.png,BlueMask.png,TealLazer.png,GrayHat.png
2,Mountains.png,Neck.png,T-Shirt.png,BikerJacket.png,Ear.png,face.png,Nose.png,Arched.png,Glasses.png,Border and text.png,SlickedHair.png,GrinningMouth.png,BlueMask.png,TealLazer.png,GrayHat.png
3,LightRed.png,Neck.png,T-Shirt.png,BikerJacket.png,Ear.png,face.png,Nose.png,Arched.png,Glasses.png,Border and text.png,SlickedHair.png,GrinningMouth.png,BlueMask.png,TealLazer.png,GrayHat.png
4,LightBlue.png,Neck.png,T-Shirt.png,BikerJacket.png,Ear.png,face.png,Nose.png,Arched.png,Glasses.png,Border and text.png,SlickedHair.png,GrinningMouth.png,BlueMask.png,TealLazer.png,GrayHat.png


In [ ]:
# test[0][1]

'LightRed.png'

In [ ]:
count_row = test.shape[0]  # Gives number of rows
count_col = test.shape[1] 
print(count_col)

15


In [ ]:
# function to overlay images
def addparts (image):
  Border = cv2.imread(image, cv2.IMREAD_UNCHANGED)

  # normalize alpha channels from 0-255 to 0-1
  alpha_background = background[:,:,3] / 255.0

  alpha_Border = Border[:,:,3] / 255.0

  # set adjusted colors
  for color in range(0, 3):
      background[:,:,color] = alpha_Border * Border[:,:,color] + \
          alpha_background * background[:,:,color] * (1 - alpha_Border)

  # set adjusted alpha and denormalize back to 0-255
  background[:,:,3] = (1 - (1 - alpha_Border) * (1 - alpha_background)) * 255
  # display the image
  #cv2_imshow(background)
  # cv2.waitKey(0)

  return background


In [ ]:
n=9001
count = 0
for i in range(count_row):
  #img1 = backgroud,test[9][i+1]
  background = cv2.imread(test[0][i+1], cv2.IMREAD_UNCHANGED)
  #img2 = face.png
  face = cv2.imread('face.png', cv2.IMREAD_UNCHANGED)

  # normalize alpha channels from 0-255 to 0-1
  alpha_background = background[:,:,3] / 255.0
  alpha_face = face[:,:,3] / 255.0

  # set adjusted colors
  for color in range(0, 3):
      background[:,:,color] = alpha_face * face[:,:,color] + \
          alpha_background * background[:,:,color] * (1 - alpha_face)

  # set adjusted alpha and denormalize back to 0-255
  background[:,:,3] = (1 - (1 - alpha_face) * (1 - alpha_background)) * 255

  for j in range(count_col):
    if test[j][i+1] == 'Blank.png':
      continue
    else:
      addparts(test[j][i+1])
      
  # display the image
  #cv2_imshow(background)

  #save image to drive
  cv2.imwrite(f'/content/drive/MyDrive/NFT 9001-11664/NFT{n}.png',background) 

  n = n+1
  cv2.waitKey(0)
  print(count)
  count = count+1




2462
2463
2464
2465
2466
2467
2468
2469
2470
2471
2472
2473
2474
2475
2476
2477
2478
2479
2480
2481
2482
2483
2484
2485
2486
2487
2488
2489
2490
2491
2492
2493
2494
2495
2496
2497
2498
2499
2500
2501
2502
2503
2504
2505
2506
2507
2508
2509
2510
2511
2512
2513
2514
2515
2516
2517
2518
2519
2520
2521
2522
2523
2524
2525
2526
2527
2528
2529
2530
2531
2532
2533
2534
2535
2536
2537
2538
2539
2540
2541
2542
2543
2544
2545
2546
2547
2548
2549
2550
2551
2552
2553
2554
2555
2556
2557
2558
2559
2560
2561
2562
2563
2564
2565
2566
2567
2568
2569
2570
2571
2572
2573
2574
2575
2576
2577
2578
2579
2580
2581
2582
2583
2584
2585
2586
2587
2588
2589
2590
2591
2592
2593
2594
2595
2596
2597
2598
2599
2600
2601
2602
2603
2604
2605
2606
2607
2608
2609
2610
2611
2612
2613
2614
2615
2616
2617
2618
2619
2620
2621
2622
2623
2624
2625
2626
2627
2628
2629
2630
2631
2632
2633
2634
2635
2636
2637
2638
2639
2640
2641
2642
2643
2644
2645
2646
2647
2648
2649
2650
2651
2652
2653
2654
2655
2656
2657
2658
2659
2660
2661


KeyError: ignored

# not used

In [ ]:
# test[0][0]
# test[1][0]
# test[2][0]

In [ ]:
# #[col][row]
# test[4][1] == 'Blank.png'

True

In [ ]:
# img = cv2.imread('RedHat.png',cv2.IMREAD_UNCHANGED)
# cv2_imshow(img)

In [ ]:
# for i in range(count_col):
#   #img1 = backgroud,test[9][i+1]
#   background = cv2.imread(test[0][i+1], cv2.IMREAD_UNCHANGED)
#   #img2 = face.png
#   face = cv2.imread('face.png', cv2.IMREAD_UNCHANGED)

#   # normalize alpha channels from 0-255 to 0-1
#   alpha_background = background[:,:,3] / 255.0
#   alpha_face = face[:,:,3] / 255.0

#   # set adjusted colors
#   for color in range(0, 3):
#       background[:,:,color] = alpha_face * face[:,:,color] + \
#           alpha_background * background[:,:,color] * (1 - alpha_face)

#   # set adjusted alpha and denormalize back to 0-255
#   background[:,:,3] = (1 - (1 - alpha_face) * (1 - alpha_background)) * 255

#   for j in range(count_row):
#     print(j)
#     if test[j][i+1] == 'Blank.png':
#       continue
#     else:
#       Border = cv2.imread(test[j][i+1], cv2.IMREAD_UNCHANGED)
#       # normalize alpha channels from 0-255 to 0-1
#       alpha_background = background[:,:,3] / 255.0

#       alpha_Border = Border[:,:,3] / 255.0

#       # set adjusted colors
#       for color in range(0, 3):
#           background[:,:,color] = alpha_Border * Border[:,:,color] + \
#               alpha_background * background[:,:,color] * (1 - alpha_Border)

#       # set adjusted alpha and denormalize back to 0-255
#       background[:,:,3] = (1 - (1 - alpha_Border) * (1 - alpha_background)) * 255
#       # display the image
#       cv2_imshow(background)
      
#       # cv2.waitKey(0)


#       # cv2.imwrite('/content/image1.png',background)

#   # display the image
#   # cv2_imshow(background)
#   # cv2.waitKey(0)

#   break

In [ ]:
# def save_image ():
#   for i in range(count_col):
#     for j in range(count_row):
#       if test[j][i+1] == 'Blank.png':
#         continue
#       else:
#         addparts(test[j][i+1])
#   return cv2.imwrite('/content/image2.png',background)

In [ ]:
image = cv2.imread('Blank.png')

In [ ]:
cv2.imwrite('/content/image.png',image)

True

In [ ]:
cv2_imshow(background)

In [ ]:
print(n)

2


In [ ]:
for i in range(count_col):
  for j in range(count_row):
    if test[j][i] == 'Blank.png':
      print(i,j,"true")
      continue
  print(i)

0
1 3 true
1 4 true
1
2 0 true
2 3 true
2 4 true
2
3 0 true
3 3 true
3 4 true
3
4 0 true
4 3 true
4 4 true
4
5 0 true
5 3 true
5 4 true
5
6 0 true
6 3 true
6 4 true
6
7 0 true
7 3 true
7 4 true
7
8 0 true
8 3 true
8 4 true
8
9 0 true
9 3 true
9 4 true
9


In [ ]:
background = cv2.imread(test[0][1], cv2.IMREAD_UNCHANGED)
cv2_imshow(background)

In [ ]:
#img1 = backgroud
background = cv2.imread(test[9][4], cv2.IMREAD_UNCHANGED)
#img2 = Nose.png
face = cv2.imread('Nose.png', cv2.IMREAD_UNCHANGED)

# normalize alpha channels from 0-255 to 0-1
alpha_background = background[:,:,3] / 255.0
alpha_face = face[:,:,3] / 255.0

# set adjusted colors
for color in range(0, 3):
    background[:,:,color] = alpha_face * face[:,:,color] + \
        alpha_background * background[:,:,color] * (1 - alpha_face)

# set adjusted alpha and denormalize back to 0-255
background[:,:,3] = (1 - (1 - alpha_face) * (1 - alpha_background)) * 255
# display the image
cv2_imshow(background)
cv2.waitKey(0)



In [ ]:
addparts ('SlickedHair.png')

In [ ]:
cv2_imshow(background)

In [ ]:
# background = cv2.imread("LightBlue.png", cv2.IMREAD_UNCHANGED)

Border = cv2.imread('SlickedHair.png', cv2.IMREAD_UNCHANGED)

# normalize alpha channels from 0-255 to 0-1
alpha_background = background[:,:,3] / 255.0

alpha_Border = Border[:,:,3] / 255.0

# set adjusted colors
for color in range(0, 3):
    background[:,:,color] = alpha_Border * Border[:,:,color] + \
        alpha_background * background[:,:,color] * (1 - alpha_Border)

# set adjusted alpha and denormalize back to 0-255
background[:,:,3] = (1 - (1 - alpha_Border) * (1 - alpha_background)) * 255

# newbackgroud = background
# display the image
cv2_imshow(background)
cv2.waitKey(0)

In [ ]:
Border = cv2.imread('RedHat.png', cv2.IMREAD_UNCHANGED)

# normalize alpha channels from 0-255 to 0-1
alpha_background = background[:,:,3] / 255.0

alpha_Border = Border[:,:,3] / 255.0

# set adjusted colors
for color in range(0, 3):
    background[:,:,color] = alpha_Border * Border[:,:,color] + \
        alpha_background * background[:,:,color] * (1 - alpha_Border)

# set adjusted alpha and denormalize back to 0-255
background[:,:,3] = (1 - (1 - alpha_Border) * (1 - alpha_background)) * 255

# newbackgroud = background
# display the image
cv2_imshow(background)
cv2.waitKey(0)

In [ ]:
background = cv2.imread("LightBlue.png", cv2.IMREAD_UNCHANGED)
foreground = cv2.imread("Coat.png", cv2.IMREAD_UNCHANGED)

# normalize alpha channels from 0-255 to 0-1
alpha_background = background[:,:,3] / 255.0
alpha_foreground = foreground[:,:,3] / 255.0

# set adjusted colors
for color in range(0, 3):
    background[:,:,color] = alpha_foreground * foreground[:,:,color] + \
        alpha_background * background[:,:,color] * (1 - alpha_foreground)

# set adjusted alpha and denormalize back to 0-255
background[:,:,3] = (1 - (1 - alpha_foreground) * (1 - alpha_background)) * 255

# display the image
cv2_imshow(background)
cv2.waitKey(0)

In [ ]:
img_1 = cv2.imread('LightBlue.png', cv2.IMREAD_UNCHANGED)
# rgba_1 = cv2.cvtColor(img_1, cv2.COLOR_RGB2RGBA)
img_2 = cv2.imread('Coat.png',cv2.IMREAD_UNCHANGED)
# rgba_2 = cv2.cvtColor(img_2, cv2.COLOR_RGB2RGBA)
# img_3 = cv2.imread('Border and text.png')
# img_4 = cv2.imread('FaceSkin.png')
# img_5 = cv2.imread('Ear.png')
# img_6 = cv2.imread('Sunglasses.png')
# img_7 = cv2.imread('GrinningMouth.png')
no_img = 2
img = rgba_1/no_img + rgba_2/no_img
#  + img_3/no_img + img_4/no_img + img_5/no_img + img_6/no_img + img_7/no_img
cv2_imshow(img)

In [ ]:
print(img_2)

In [ ]:
# read two images
# src1 = cv2.imread('image1.png', cv2.IMREAD_COLOR)
# src2 = cv2.imread('image2.png', cv2.IMREAD_COLOR)
img_1 = cv2.imread('NuclearPlant.png')

img_1_lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)  # Convert color space from BGR to LAB color space

img_2 = cv2.imread('Border and text.png')

img_2_lab = cv2.cvtColor(img_2, cv2.COLOR_BGR2LAB)  # Convert color space from BGR to LAB color space

# print(img_2.shape)

# img_1 = cv2.resize(img_1, (400, 400))

# blending
#blend = cv2.addWeighted(img, 0.7, scr2, 0.3, 0)
blend_lab = cv2.addWeighted(img_1_lab, 0.7, img_2_lab, 0.3, 0)  # Blend images in LAB color space
blend = cv2.cvtColor(blend_lab, cv2.COLOR_LAB2BGR)  # Convert color space from LAB to BGR color space

# # add or blend the images
# img = cv2.addWeighted(img_1, 1, img_2, 2, 0.0)

# Replace blended "background" white pixels of scr2 with original pixel values from img:
mask = np.all(img_2 == 255, 2).astype(np.uint8)  # Mask value is 1 where 3 color channels are 255
cv2.copyTo(img_1, mask, blend)  # Copy pixels from img to blend where mask != 0.

# Show the results

cv2_imshow(blend)



In [ ]:
cv2_imshow(img)

In [ ]:
img_1

In [ ]:
for i in range(3):
  for j in range(5):
    print("j:",j)
  print(i)